# Symbolic Expressions

## Contents

+ Modeling algebraic expressions as data structures.
+ Writing code to analyze, transform, or evaluate algebraic expressions.
+ Finding the derivative of a function by manipulating the expression that defines it.
+ Writing a Python function to compute derivative formulas.
+ Using [SymPy](https://www.sympy.org/en/index.html) to compute integral formulas.

## Intro

The previous two chapters introduces two of the most important concepts in calculus: the derivative and the integral.

We saw that you can approximate the derivative of a function at a point by taking slopes of smaller and smaller secant lines. We also saw that you can approximate the integral of a function by estimating the area under the graph with skinnier and skinnier rectangles.

Up until now we've worked with approximations, because we were relying on the capacity of Python running on a computer, but Calculus teaches us how to obtain exact values for the derivative and integral.

For example, if $ f(x) = x^3 $, Calculus tells us that:

$
f'(x) = 3 \cdot x^2
$

There are formulas for each type of function, that you would apply when asked for the exact derivative of a function.

However, as developers, knowing the formulas is not a very interesting skill for us. Instead, we can realy on a specialized tool called *computer algebra system* to compute it for us.

## Finding an exact derivative with a computer algebra system

Mathematica is one of the most popular computer algebra systems. Its engine is free only at [Wolfram Alpha](https://www.wolframalpha.com/).

This site can be used to find the exact formula for a derivative.

For example, when building neural networks, it's useful to know the derivative of the function:

$
\displaystyle
f(x) = \frac{1}{1 + e^{-x}}
$

This can be easily obtained by entering that formula in Python syntax. After pressing enter, it will display several facts about the function, including its derivative:

![Entering formula](pics/wolfram-alpha-input-formula.png)

![Derivative and Integral](pics/wolfram-alpha-derivative-integral.png)

Mathematica does not rely on the approximations we've worked on the previous chapters. Instead, Wolfram Alpha interprets the formula you've typed in, transforms it with some algebraic manipulations, and outputs a new formula.

This is called **symbolic programming**.

### Doing symbolic algebra in Python

Let's start by illustrating how we will be representing and manipulating formulas in Python.

Let's assume we have a mathematical function like:

$
f(x) = (3 x^2  + x) \cdot sin(x)
$

We already know that the best way to represent the formula in order to evaluate it would be:

```python
from math import sin

def f(x):
  return (2 * x ** 2 + x) * sin(x)
```

But that representation is not helpful to learn facts about the formula, such as:

+ Does the formula depend on the variable x?
+ Does it contain a trigonometric function?
+ Does it involve the operation of division?

While we can really easily answer those questions just by looking at the function, the Python representation used to get the function values wouldn't help.

For example, it'd be very difficult to write a function `contains_division(f)` that returns true if it uses the operation division in its definition.

Thus, we need to find another way to represent the function. We need another way to express the function that would tell us what operations are being applied and in what order.

For instance, the function $ f(x) $ above is a product of $ sin(x) $ with a sum. We also know that there's a well-known algebraic process for expanding the function definition:

$
f(x) = (3 x^2  + x) \cdot sin(x) = 3 x^2 \cdot sin(x) + x \cdot sin(x)
$

The new way to represent the formula should let us apply those rules as well.

In summary, we need to *model algebraic expressions as data structures* so that we can manipulate them symbolically, and therefore automate the rules of calculus.

Once we have that, we will be able to calculate derivatives.

Most functions expressed by simple formulas also have simple formulas for their derivatives. For example, if $ f(x) = x^3 $, $ f'(x) = 3 \cdot x^2 $.

That means that the derivative of $ x^3 $ at any value of $ x $ is $ 3 \cdot x^2 $.

Another requirement for our data structures is that we will have to be able to calculate the derivative or any given algebraic expression.

At the minimum, we will need to represent variables, numbers, sums, differences, products, quotients, powers, and functions such as sine and cosine and take the derivatives of them.

## Modeling algebraic expressions

Let's develop our intuition around how we can break an algebraic expression such as $ f(x) = (3 \cdot x^2 + x) \cdot sin(x) $.

It contains the following building blocks we should model:
+ a variable $ x $
+ numbers ($ 3 $)
+ operations: addition, multiplication, power
+ a named function: $ sin(x) $

The goal is to translate it into a Python data structure we can work with.

The first observation is that $ f $ is an arbitrary name. The right hand side will evaluate the same whether the function is called $ f $, $ g $, or $ h $.

As a result, we should be focusing on the right hand side of the function definition, which is called an expression:
> An expression is a collection of mathematical symbols combined in some valid ways.

### Breaking an expression into pieces

The way in which we can model algebraic expressions such as $ (3 \cdot x^2 + x) \cdot sin(x) $ is to break them into smaller expressions.

There is only one meaningful way to break up the expressions $ (3  \cdot x^2 + x) \cdot sin(x) $. Namely, it's the product of $ (3 \cdot x^2 + x) $ and $ sin(x) $.

Other ways of breaking up the expression will end up in something we can't make sense of according to mathematical rules:

![Breaking up expressions](pics/breaking_expressions.png)

If we apply the same strategy to the expression $ 3 \cdot x^2 + x $ we will see that it can be broken down as:

![expression tree](pics/expression_tree.png)

If we inspect the approach we see that we take:
+ operations &mdash; ways to take two or more algebraic expressions and stick them together side by side to make a new, bigger algebraic expression.
+ operators &mdash; valid places to break up an existing algebraic expression into smaller ones.


In the terminology of functional programming, functions combining smaller objects into bigger ones like this are often called *combinators*.

Some of the combinators we've used in the expression above are:
+ $ 3 \cdot x^2 $ is the product of the expression 3 and $ x^2 $
+ $ x^2 $ is a power: one expression $ x $ raise to the power of another expression $ 2 $.
+ $ sin(x) $ is a *function application*. Given the expression $ sin $ and the expression $ x $, we can build a new expression $ sin(x) $.

By contrast, a variable $ x $, a number $ 2 $, or a function named $ sin $ cannot be broken down further, and we call them *elements*.

### Building an expression tree